In [ ]:
!nvidia-smi

Fri Jul 30 20:17:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Importing 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install transformers -q
# !pip install tensorflow_text -q

In [ ]:
import os
import csv

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from sklearn.preprocessing import LabelEncoder

## Config

In [ ]:
BATCH_SIZE = 256
#SEQ_LEN = 200

## Loading Data

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/AmazonML/input/new_train.csv', usecols=['TITLE', 'BULLET_POINTS', 'label3', 'split3'],)
train.fillna('nan', inplace=True)

In [ ]:
train.head(2)

,TITLE,BULLET_POINTS,label3,split3
0,BlingTon Brown Wallet/Purse for Men and Boys,"[Coin Pocket,2 Compartments,2 Card Slots,2 Inn...",816,valid
1,"Nestle NesPlus Breakfast Cereal, Multigrain Gr...","[A tasty, nutritious breakfast cereal for happ...",65,train


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/AmazonML/input/dataset/test.csv', escapechar='\\', quoting=csv.QUOTE_NONE)
test.head(2)

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal


In [ ]:
np.sum(test.isna())

PRODUCT_ID           0
TITLE                7
DESCRIPTION      10469
BULLET_POINTS     6786
BRAND             2430
dtype: int64

In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/AmazonML/input/dataset/sample_submission.csv')
sample.head(2)

,PRODUCT_ID,BROWSE_NODE_ID
0,1,0
1,2,1


In [ ]:
np.sum(sample.isna())

PRODUCT_ID        0
BROWSE_NODE_ID    0
dtype: int64

## Creating dataset

In [ ]:
le = LabelEncoder()
train['label3'] = le.fit_transform(train['label3'])

train_df, valid_df = train[train['split3'] == 'train'], train[train['split3'] == 'valid']
train_df['text'] = train_df['TITLE'] + " " + train_df['BULLET_POINTS']
valid_df['text'] = valid_df['TITLE'] + " " + valid_df['BULLET_POINTS']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((train_df['text'], train_df['label3']))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(AUTO)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_df['text'], valid_df['label3']))
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
for i in train_dataset:
  print(i[0].shape, i[1].shape)
  break

(256,) (256,)


## Model

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/2", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
hidden = tf.keras.layers.Dense(1024, activation='relu')(pooled_output)
classifier = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden)

model = tf.keras.Model(inputs=text_input, outputs=classifier)

In [ ]:
model.load_weights('/content/drive/MyDrive/AmazonML/label3model_weights.h5')

### Hide

In [ ]:
# seq_length = SEQ_LEN

# # text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string), tf.keras.layers.Input(shape=(), dtype=tf.string)]
# text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

# # preprocessor     = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# # tokenize         = hub.KerasLayer(preprocessor.tokenize)
# # bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))
# encoder          = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", 
#                                   trainable=True, 
#                                   output_key='pooled_output')

# preprocessor    = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# encoder_inputs  = preprocessor(text_input)

# # tokenized_inputs = [tokenize(segment) for segment in text_inputs]
# # encoder_inputs   = bert_pack_inputs(tokenized_inputs)
# encoder_outputs  = encoder(encoder_inputs)
# #pooled_outputs   = encoder_outputs["pooled_output"]
# hidden_layer     = tf.keras.layers.Dense(1024, activation='relu')(encoder_outputs)
# classifier       = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)

# model = tf.keras.Model(inputs=text_inputs, outputs=classifier)

## Training

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['acc'])

In [ ]:
# sample_pred = None
# for i in train_dataset:
#   print(i)
#   sample_pred = model(i[0])
#   break

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/AmazonML/label3model_new.h5',
                                              save_best_only=True, monitor='acc', 
                                              mode='max', save_weights_only=True,
                                              save_freq=500)

In [ ]:
# model.save('/content/drive/MyDrive/AmazonML/label3model.h5')
# model.save_weights('/content/drive/MyDrive/AmazonML/label3model_weights.h5')

In [ ]:
model.fit(train_dataset,
          epochs=2,
          validation_data=valid_dataset,
          callbacks = [ckpt])